In [1]:
import glob
import os
import hashlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3

In [2]:
def get_md5sum(x):
    return hashlib.md5(x.encode("utf-8")).hexdigest()[:10]

In [3]:
def map_assembler(cell):
    if "carpedeam" in cell:
        return "CarpeDeam"
    elif "penguin" in cell:
        return "PenguiN"
    elif "megahit" in cell:
        return "MEGAHIT"
    elif "spades" in cell:
        return "metaSPAdes"
    else:
        return cell  # Return the cell as is if none of the conditions are met

def adjust_assemblerconfig(row):
    if row["assembler"] == "CarpeDeam":
        if "Safe" in row["file"]:
            return "CarpeDeam (safe mode)"
        elif "Unsafe" in row["file"]:
            return "CarpeDeam (unsafe mode)"
        else:
            return "CarpeDeam\n(safe mode)"
    else:
        return row["assembler"]

In [4]:
labels = ["ERR3579753", "ERR3579736"]

labels_dict = {key: get_md5sum(key) for key in labels}
labels_dict_inv = {value: key for key, value in labels_dict.items()}
print(labels_dict_inv)

{'eebf379d54': 'ERR3579753', '87bf691987': 'ERR3579736', 'b9fca77ede': 'ERR3579731', '7290ed61ac': 'ERR3579732'}


In [5]:
sample_map = {'87bf691987' : 'EMN001', 'eebf379d54' : 'GDN001'}

In [6]:
def find_files(directory, suffix):
    aln_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(suffix):
                aln_files.append(os.path.join(root, file))
    return aln_files

In [26]:
def plot_unique_hits(dfs, titles, dataset, number):


    assembler_order=["CarpeDeam (safe mode)", "CarpeDeam (unsafe mode)", "MEGAHIT", "PenguiN", "metaSPAdes"]
    assembler_colors= ['#a1c9f4', '#b9f2f0', '#ffb482','#8de5a1','#ff9f9b']

    # Ensure the input is a list of DataFrames and titles
    if not isinstance(dfs, list) or not all(isinstance(df, pd.DataFrame) for df in dfs):
        raise ValueError("Input must be a list of pandas DataFrames.")

    # Extract unique hits per assembler
    unique_hits = [df['reference'].nunique() for df in dfs]

    # Create a data frame for plotting
    data = pd.DataFrame({'Assembler': titles, 'Unique Hits': unique_hits})

    # Set the order of assemblers if provided
    if assembler_order:
        data = data.set_index('Assembler').reindex(assembler_order).reset_index()

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot horizontal bar chart using seaborn
    sns.barplot(x='Unique Hits', y='Assembler', data=data, palette=assembler_colors, ax=ax)

    # Rotate x-axis labels
    ax.set_xlabel('Number of Unique Hits', fontsize=12)
    ax.set_ylabel('Assembler', fontsize=12)

    # Add values to bars
    for i, v in enumerate(data['Unique Hits']):
        if not pd.isna(v):
            ax.text(v + 0.1, i, str(int(v)), va='center', fontsize=12)
        else:
            ax.text(0.1, i, 'NaN', va='center', fontsize=12)

    # Adjust layout
    plt.tight_layout()

    plt.savefig(f'plots/unique_hits_{dataset}_{number}.svg', format="svg", bbox_inches='tight')
    plt.show()

In [ ]:
def curate_report_df(file):
    """
    Returns a list of dataframes. Each dataframe belongs to a file/assembler. The analyzed files are from mmseq taxonomy:
    (1) Query
    (2) Target
    (3) Seq.Id.
    (4) Alignment Length
    (5) Number of mismatches
    (6) number of gap openings
    (7) Start in Query
    (8) End in Query
    (9) Start in Target
    (10) End in Target
    (11) Eval
    (12) bit score
    """
    
    df_aln = pd.read_csv(file, sep='\t',names=["query", "target", "seq.Id.", "alnLen", "MM", "gaps", "startQuery", "EndQuery", "startTarget", \
                                                "EndTarget", "Eval", "bit score", "queryLen", "targetLen", "queryCov", "targetCov"])
    df_aln["reference"] = df_aln["target"].str.rsplit('_', n=1).str[1]

    df_aln = df_aln[ df_aln["seq.Id."] >= 0.35 ]
    df_aln = df_aln[ df_aln["Eval"] <= 1e-12 ]
    df_aln = df_aln[ df_aln["alnLen"] >= 100 ]
    df_aln = df_aln.sort_values(by=["Eval"], ascending=True)
    df_aln = df_aln.drop_duplicates(subset=["query"], keep='first')
    df_aln = df_aln.drop_duplicates(subset=["reference"], keep='first')

    df_aln.reset_index(inplace=True)
    
    df_aln["file"] = os.path.basename(file)
    df_aln["dataLabel"]=df_aln["file"].str.split(".").str[0].map(sample_map)
    df_aln["assembler"] = df_aln["file"].apply(lambda file: map_assembler(file))
    df_aln["assembler_final"] = df_aln.apply(adjust_assemblerconfig, axis=1)

    #print(df_aln)
    
    return df_aln

In [ ]:
### TOPHIT REPORT ANALYSIS

assemblers = ['carpedeam2.configSafe', 'carpedeam2.configUnsafe', 'megahit.config0', 'penguin.config0', 'spades.config0']
samples = ["GDN001", "EMN001"]

dic = {value: {} for value in samples}
print(dic)
for sample in samples:
    path=f"data/{sample}/results/assembly-mmseqs"

    files_aln = find_files(path, ".tsv")
    for file in files_aln:
        if any(assembler in file for assembler in assemblers):
            results = curate_report_df(file)
            readname = results["dataLabel"][0]
            assembler_id = results["assembler_final"][0]
            print(assembler_id)
            dic[readname][assembler_id] = results

In [ ]:
for dataset in dic.keys():
    plot_unique_hits([dic[dataset]["CarpeDeam (safe mode)"], dic[dataset]["CarpeDeam (unsafe mode)"], dic[dataset]["MEGAHIT"], dic[dataset]["PenguiN"], dic[dataset]["metaSPAdes"]], ["CarpeDeam (safe mode)", "CarpeDeam (unsafe mode)", "MEGAHIT", "PenguiN", "metaSPAdes"], dataset, 1)